In [660]:
# Import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import talib as ta
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
import datetime


In [661]:
df = pd.read_csv("Data-Nifty-2005-25.csv")

In [662]:
for i in ['Open','High','Low','Close']:
    df[i]  = df[i].pct_change()*100
df

,Date,Close,High,Low,Open
0,2007-09-17,NaN,NaN,NaN,NaN
1,2007-09-18,1.146926,0.060452,-0.029006,-0.538904
2,2007-09-19,4.094626,4.112663,1.532956,1.249414
3,2007-09-20,0.321187,0.461070,3.755835,4.056922
4,2007-09-21,1.895715,1.992293,0.265831,0.382274
...,...,...,...,...,...
4395,2025-08-19,0.416856,-0.037366,0.084898,-0.187863
4396,2025-08-20,0.279818,0.304041,0.224130,0.299105
4397,2025-08-21,0.132529,0.258886,0.502217,0.705762
4398,2025-08-22,-0.851748,-0.273522,-0.781284,-0.309640


In [663]:
df['Date'] = df['Date'].astype('datetime64[ns]')
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.weekday+1
df['Week'] = df['Date'].dt.isocalendar().week
df.set_index('Date',inplace=True)
df

,Close,High,Low,Open,Month,Weekday,Week
Date,,,,,,,
2007-09-17,NaN,NaN,NaN,NaN,9,1,38
2007-09-18,1.146926,0.060452,-0.029006,-0.538904,9,2,38
2007-09-19,4.094626,4.112663,1.532956,1.249414,9,3,38
2007-09-20,0.321187,0.461070,3.755835,4.056922,9,4,38
2007-09-21,1.895715,1.992293,0.265831,0.382274,9,5,38
...,...,...,...,...,...,...,...
2025-08-19,0.416856,-0.037366,0.084898,-0.187863,8,2,34
2025-08-20,0.279818,0.304041,0.224130,0.299105,8,3,34
2025-08-21,0.132529,0.258886,0.502217,0.705762,8,4,34


In [664]:
df = df[['Close','Month','Weekday','Week']]
df['Close'] = abs(df['Close'])

/tmp/ipykernel_5079/2157224752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close'] = abs(df['Close'])


In [665]:
data1 = df[['Close','Week']]
data1 = data1.groupby(by='Week')['Close'].agg(['mean','std'])
data1

,mean,std
Week,,
1,0.690833,0.682977
2,0.716105,0.874971
3,0.893944,0.862077
4,1.165920,1.540684
5,1.012049,0.919563
6,0.808997,0.812889
7,0.884864,0.970790
8,0.839404,0.907023
9,0.899966,0.807777


In [666]:
nlargest_mean = pd.DataFrame()
nlargest_std = pd.DataFrame()
nsmallest_mean = pd.DataFrame()
nsmallest_std = pd.DataFrame()



In [667]:
k=data1['mean'].nlargest(10).keys()
v = data1['mean'].nlargest(10).values
r = [i for i in range(1,11)]
nlargest_mean['weekno'] = k
nlargest_mean['Change'] = v
nlargest_mean.index = [i for i in range(1,11)]
nlargest_mean


,weekno,Change
1,13,1.174900
2,4,1.165920
3,11,1.077166
4,41,1.068873
5,21,1.060367
6,43,1.014745
7,12,1.013342
8,38,1.012905
9,5,1.012049
10,14,0.992528


In [668]:
k=data1['mean'].nsmallest(10).keys()
v = data1['mean'].nsmallest(10).values
r = [i for i in range(1,11)]
nsmallest_mean['weekno'] = k
nsmallest_mean['Change'] = v
nsmallest_mean.index = [i for i in range(1,11)]
nsmallest_mean


,weekno,Change
1,53,0.417559
2,1,0.690833
3,34,0.712413
4,32,0.715774
5,2,0.716105
6,24,0.725308
7,31,0.727533
8,27,0.732766
9,49,0.740757
10,22,0.740983


In [669]:
k=data1['std'].nlargest(10).keys()
v = data1['std'].nlargest(10).values
r = [i for i in range(1,11)]
nlargest_std['weekno'] = k
nlargest_std['Change'] = v
nlargest_std.index = [i for i in range(1,11)]
nlargest_std


,weekno,Change
1,21,1.956233
2,13,1.809243
3,43,1.651140
4,4,1.540684
5,12,1.298749
6,11,1.267954
7,15,1.246672
8,42,1.229565
9,46,1.159830
10,44,1.144038


In [670]:
k=data1['std'].nsmallest(10).keys()
v = data1['std'].nsmallest(10).values
r = [i for i in range(1,11)]
nsmallest_std['weekno'] = k
nsmallest_std['Change'] = v
nsmallest_std.index = [i for i in range(1,11)]
nsmallest_std


,weekno,Change
1,53,0.281716
2,31,0.627637
3,32,0.630566
4,17,0.634469
5,1,0.682977
6,34,0.686369
7,48,0.710187
8,52,0.710706
9,24,0.721828
10,26,0.726421
